# Parkway Project Use Case 1: Write Off Cases Prediction

## DATA PREPARATION

### Load Data

#### Import packages

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_rows', 500)
import warnings
warnings.filterwarnings(action='once')
import pickle
from imblearn.over_sampling import SMOTE

In [2]:
import os
os.chdir('d:/Users/mokky/Documents/GitHub/nus-iss/PRS-PM-ISY5002-GROUP5/SystemCode')
print(os.getcwd())

d:\Users\mokky\Documents\GitHub\nus-iss\PRS-PM-ISY5002-GROUP5\SystemCode


In [3]:
%load_ext autoreload
%autoreload 2
from datapipeline import Datapipeline
dpl = Datapipeline()

#### Read data from file

In [4]:
file_dir = './data/uc1'
dict_X_train_file_paths = {
    'GHL' : [file_dir + 'GHL_new_train_X_uc1.pkl'],
    'MEH' : [file_dir + 'MEH_new_train_X_uc1.pkl'],
    'PEH' : [file_dir + 'PEH_new_train_X_uc1.pkl'],
    'PNH' : [file_dir + 'PNH_new_train_X_uc1.pkl']
}
dict_y_train_file_paths = {
    'GHL' : file_dir + 'GHL_data_y_train.pkl',
    'MEH' : file_dir + 'MEH_data_y_train.pkl',
    'PEH' : file_dir + 'PEH_data_y_train.pkl',
    'PNH' : file_dir + 'PNH_data_y_train.pkl'
}
dict_X_test_file_paths = {
    'GHL' : [file_dir + 'GHL_new_test_X_uc1.pkl'],
    'MEH' : [file_dir + 'MEH_new_test_X_uc1.pkl'],
    'PEH' : [file_dir + 'PEH_new_test_X_uc1.pkl'],
    'PNH' : [file_dir + 'PNH_new_test_X_uc1.pkl']
}
dict_y_test_file_paths = {
    'GHL' : file_dir + 'GHL_data_y_test.pkl',
    'MEH' : file_dir + 'MEH_data_y_test.pkl',
    'PEH' : file_dir + 'PEH_data_y_test.pkl',
    'PNH' : file_dir + 'PNH_data_y_test.pkl'
}

In [5]:
dict_df_X_train = {}
dict_df_y_train = {}
dict_df_X_test = {}
dict_df_y_test = {}

for hosp in dict_X_train_file_paths:
    dict_df_X_train[hosp] = pd.concat([pd.read_pickle(file_path)
                                       for file_path in dict_X_train_file_paths[hosp]])
    print(f'X_train {hosp} {dict_df_X_train[hosp].shape}')
    
    dict_df_y_train[hosp] = pd.read_pickle(dict_y_train_file_paths[hosp])
    print(f'y_train {hosp} {dict_df_y_train[hosp].shape}')

    dict_df_X_test[hosp] = pd.concat([pd.read_pickle(file_path)
                                      for file_path in dict_X_test_file_paths[hosp]])
    print(f'X_test {hosp} {dict_df_X_test[hosp].shape}')

    dict_df_y_test[hosp] = pd.read_pickle(dict_y_test_file_paths[hosp])
    print(f'y_test {hosp} {dict_df_y_test[hosp].shape}')

FileNotFoundError: [Errno 2] No such file or directory: './data/uc1GHL_new_train_X_uc1.pkl'

In [ ]:
def convert_y(df):
    df1 = dpl.bin_column(df.to_frame(), col='WRITE_OFF', bin_thresh = [500])
    df1 = pd.get_dummies(df1['bin_WRITE_OFF'])
    df1 = df1.drop(0, axis=1)
    df1.columns = ['WRITE_OFF_LABEL']
    return df1

In [ ]:
for hosp in dict_df_y_train:
    dict_df_y_train[hosp] = convert_y(dict_df_y_train[hosp])
    dict_df_y_test[hosp] = convert_y(dict_df_y_test[hosp])

## DECISION TREE

### Decision Tree WITHOUT SMOTE

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.tree import export_graphviz
from subprocess import call

In [ ]:
criterion = 'entropy'
rand_seed = 0

In [ ]:
for hosp in dict_df_X_train:
    model = DecisionTreeClassifier(criterion=criterion,random_state=rand_seed)
    model.fit(dict_df_X_train[hosp], dict_df_y_train[hosp])
    print(model)
    print(f"Accuracy on {hosp} training set without SMOTE decision tree: {model.score(dict_df_X_train[hosp], dict_df_y_train[hosp])}")
    print(f"Accuracy on {hosp} test set without SMOTE decision tree: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")
    
    filename = file_dir + f'{hosp}_uc1_no_smote_dt_model.pkl'
    pickle.dump(model, open(filename, 'wb'))
    
    y_pred = model.predict(dict_df_X_test[hosp])
    print(hosp, 'decision tree\n', confusion_matrix(dict_df_y_test[hosp], y_pred))  
    print(hosp, 'decision tree\n', classification_report(dict_df_y_test[hosp], y_pred)) 

### Decision Tree WITH SMOTE

#### Decision Tree WITH SMOTE SS = 0.1 ~ 1.0

In [ ]:
for hosp in dict_df_X_train:
    for sample in np.arange(0.1, 1.1, 0.1):
        sm_ss = SMOTE(random_state=55,sampling_strategy=sample)
        X_train_ss, y_train_ss = sm_ss.fit_sample(dict_df_X_train[hosp], dict_df_y_train[hosp])
        
        model = DecisionTreeClassifier(criterion=criterion,random_state=rand_seed)
        model.fit(X_train_ss, y_train_ss)
        print(model)
        print(f"Accuracy on {hosp} training set with {sample} SMOTE decision tree: {model.score(X_train_ss, y_train_ss)}")
        print(f"Accuracy on {hosp} test set with {sample} SMOTE decision tree: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")
    
        filename = file_dir + f'{hosp}_uc1_{sample}_smote_dt_model.pkl'
        pickle.dump(model, open(filename, 'wb'))
        
        y_pred = model.predict(dict_df_X_test[hosp])
        print(hosp, 'decision tree \n', confusion_matrix(dict_df_y_test[hosp], y_pred))  
        print(hosp, 'decision tree \n', classification_report(dict_df_y_test[hosp], y_pred)) 

## LOGISTIC REGRESSION

### Logistic Regression Without SMOTE

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
reg_strength = 0.01

In [ ]:
for hosp in dict_df_X_train:
    model = LogisticRegression(C=reg_strength).fit(dict_df_X_train[hosp], dict_df_y_train[hosp])
    print(model)
    print(f"{hosp} Training set without SMOTE log reg score: {model.score(dict_df_X_train[hosp], dict_df_y_train[hosp])}")
    print(f"{hosp} Test set without SMOTE log reg score: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")
    
    filename = file_dir + f'{hosp}_uc1_no_smote_logreg_model.pkl'
    pickle.dump(model, open(filename, 'wb'))
    
    y_pred = model.predict(dict_df_X_test[hosp])
    print(hosp, 'log reg \n', confusion_matrix(dict_df_y_test[hosp], y_pred))  
    print(hosp, 'log reg \n', classification_report(dict_df_y_test[hosp], y_pred)) 

### Logistic Regression With SMOTE

#### Logistic Regression With SMOTE ss=0.1 ~ 1.0

In [ ]:
for hosp in dict_df_X_train:
    for sample in np.arange(0.1, 1.1, 0.1):
        sm_ss = SMOTE(random_state=55,sampling_strategy=sample)
        X_train_ss, y_train_ss = sm_ss.fit_sample(dict_df_X_train[hosp], dict_df_y_train[hosp])
        
        model = LogisticRegression(C=reg_strength).fit(X_train_ss, y_train_ss)
        print(model)
        print(f"{hosp} Training set with {sample} SMOTE log reg score: {model.score(X_train_ss, y_train_ss)}")
        print(f"{hosp} Test set with {sample} SMOTE log reg score: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

        filename = file_dir + f'{hosp}_uc1_{sample}_smote_logreg_model.pkl'
        pickle.dump(model, open(filename, 'wb'))

        y_pred = model.predict(dict_df_X_test[hosp])
        print(hosp, 'log reg \n', confusion_matrix(dict_df_y_test[hosp], y_pred))  
        print(hosp, 'log reg \n', classification_report(dict_df_y_test[hosp], y_pred)) 

## NAIVE BAYES

### Naive Bayes Without SMOTE

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
for hosp in dict_df_X_train:
    # Initiating the Gaussian Classifier
    model = GaussianNB()

    # Training your model 
    model.fit(dict_df_X_train[hosp], dict_df_y_train[hosp])
    print(model)
    filename = file_dir + f'{hosp}_uc1_no_smote_nb_model.pkl'
    pickle.dump(model, open(filename, 'wb'))

    # Score
    print(f"{hosp} Training set without SMOTE Naive Bayes score: {model.score(dict_df_X_train[hosp], dict_df_y_train[hosp])}")
    print(f"{hosp} Test set without SMOTE Naive Bayes score: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

    # Confusion Matrix
    y_pred = model.predict(dict_df_X_test[hosp])
    print(hosp, 'Naive Bayes \n', confusion_matrix(dict_df_y_test[hosp], y_pred))  
    print(hosp, 'Naive Bayes \n', classification_report(dict_df_y_test[hosp], y_pred)) 

### Naive Bayes With SMOTE

#### Naive Bayes Without SMOTE ss=0.1 ~ 1.0

In [ ]:
for hosp in dict_df_X_train:
    for sample in np.arange(0.1, 1.1, 0.1):
        sm_ss = SMOTE(random_state=55,sampling_strategy=sample)
        X_train_ss, y_train_ss = sm_ss.fit_sample(dict_df_X_train[hosp], dict_df_y_train[hosp])
        
        # Initiating the Gaussian Classifier
        model = GaussianNB()
        
        # Training your model 
        model.fit(X_train_ss, y_train_ss)
        print(model)
        filename = file_dir + f'{hosp}_uc1_{sample}_smote_nb_model.pkl'
        pickle.dump(model, open(filename, 'wb'))

        # Score
        print(f"{hosp} Training set with {sample} SMOTE Naive Bayes score: {model.score(X_train_ss, y_train_ss)}")
        print(f"{hosp} Test set with {sample} SMOTE Naive Bayes score: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

        # Confusion Matrix
        y_pred = model.predict(dict_df_X_test[hosp])
        print(hosp, 'Naive Bayes\n', confusion_matrix(dict_df_y_test[hosp], y_pred))  
        print(hosp, 'Naive Bayes\n', classification_report(dict_df_y_test[hosp], y_pred)) 

## NEURAL NET MLP

### Neural Net MLP Without SMOTE, 2 Layers

In [ ]:
from sklearn.neural_network import MLPClassifier  
from sklearn import metrics

In [ ]:
list_hid_layers = [(128,128), (16,16,16), (32,32,32), (16,16,16,16), (32,32,32,32)]
max_iter = 1000
list_smote_sampling = [0.1, 0.2, 0.3, 0.5, 1.0]

#### Neural Net MLPClassifier without SMOTE

In [ ]:
for hosp in dict_df_X_train:
    for hid_layers in list_hid_layers:
        model = MLPClassifier(hidden_layer_sizes=hid_layers, max_iter=max_iter,verbose=1)  
        model.fit(dict_df_X_train[hosp], dict_df_y_train[hosp])
        print(model)
        filename = file_dir + f'{hosp}_uc1_no_smote_mlp_{hid_layers}_model.pkl'
        pickle.dump(model, open(filename, 'wb'))

        predictions = mlp.predict(dict_df_X_test[hosp]) 

        # Score
        print(f"{hosp} Training set without SMOTE mlp {hid_layers} score: {model.score(dict_df_X_train[hosp], dict_df_y_train[hosp])}")
        print(f"{hosp} Test set without SMOTE mlp {hid_layers} score: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

        print(f"{hosp} Accuracy without SMOTE mlp {hid_layers}: ", metrics.accuracy_score(dict_df_y_test[hosp], predictions))
        print(hosp, f'mlp {hid_layers}\n', confusion_matrix(dict_df_y_test[hosp],predictions))  
        print(hosp, f'mlp {hid_layers}\n', classification_report(dict_df_y_test[hosp],predictions))  

### Neural Net With SMOTE

In [ ]:
list_hid_layers = [(16,16), (32,32), (64,64), (128,128), (256,256),
                   (16,16,16), (32,32,32), (64,64,64),
                   (16,16,16,16), (32,32,32,32)]
max_iter = 1000
list_smote_sampling = [0.1, 0.2, 0.3, 0.5, 1.0]

#### Neural Net Layers With SMOTE

In [ ]:
for hosp in dict_df_X_train:
    for hid_layers in list_hid_layers:
        for sample in list_smote_sampling:
            sm_ss = SMOTE(random_state=55,sampling_strategy=sample)
            X_train_ss, y_train_ss = sm_ss.fit_sample(dict_df_X_train[hosp], dict_df_y_train[hosp])
        
            model = MLPClassifier(hidden_layer_sizes=hid_layers, max_iter=max_iter,verbose=1)  
            model.fit(X_train_ss, y_train_ss)  
            print(model)
            filename = file_dir + f'{hosp}_uc1_{sample}_smote_mlp_{hid_layers}_model.pkl'
            pickle.dump(model, open(filename, 'wb'))
            
            predictions = model.predict(dict_df_X_test[hosp]) 

            # Score
            print(f"{hosp} Training set {sample} SMOTE mlp {hid_layers} score: {model.score(X_train_ss, y_train_ss)}")
            print(f"{hosp} Test set {sample} SMOTE mlp {hid_layers} score: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

            print(f"{hosp} Accuracy: ", metrics.accuracy_score(dict_df_y_test[hosp], predictions))
            print(hosp, f'mlp {hid_layers}\n', confusion_matrix(dict_df_y_test[hosp],predictions))  
            print(hosp, f'mlp {hid_layers}\n', classification_report(dict_df_y_test[hosp],predictions))  

## KNN

In [ ]:
# Be careful when running this, about 2 full days are needed to run this!!!
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
for hosp in dict_df_X_train:
    test_scores = []
    train_scores = []

    for i in range(1,15):

        knn = KNeighborsClassifier(i)
        knn.fit(dict_df_X_train[hosp],dict_df_y_train[hosp])

        train_scores.append(knn.score(dict_df_X_train[hosp],dict_df_y_train[hosp]))
        test_scores.append(knn.score(dict_df_X_test[hosp],dict_df_y_test[hosp]))

    ## score that comes from testing on the same datapoints that were used for training
    max_train_score = max(train_scores)
    train_scores_ind = [i for i, v in enumerate(train_scores) if v == max_train_score]
    print(f'{hosp} Max train score {max_train_score*100} % and k = {list(map(lambda x: x+1, train_scores_ind))}')

    ## score that comes from testing on the datapoints that were split in the beginning to be used for testing solely
    max_test_score = max(test_scores)
    test_scores_ind = [i for i, v in enumerate(test_scores) if v == max_test_score]
    print(f'{hosp} Max test score {max_test_score*100} % and k = {list(map(lambda x: x+1, test_scores_ind))}')

    plt.figure(figsize=(12,5))
    plt.plot(range(1,15),train_scores,marker='*',label='Train Score')
    plt.plot(range(1,15),test_scores,marker='o',label='Test Score')
    plt.title(f'{hosp} KNN')
    plt.tight_layout()
    plt.show()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
# From the above, the best KNN is at k=8
#Setup a knn classifier with k neighbors
best_knn = KNeighborsClassifier(8)

### KNN without SMOTE

In [ ]:
n_neighbours = 8
for hosp in dict_df_X_train:
    # Fit the KNN model with training data and score the test data
    best_knn = KNeighborsClassifier(n_neighbours)
    
    best_knn.fit(dict_df_X_train[hosp],dict_df_y_train[hosp])
    best_knn.score(dict_df_X_test[hosp],dict_df_y_test[hosp])
    
    # save the model to disk
    filename = f'{hosp}_best_knn_without_smote.pkl'
    pickle.dump(best_knn, open(filename, 'wb'))
    
    y_pred = best_knn.predict(dict_df_X_test[hosp])
    # Score for Best KNN, No SMOTE
    print(f"{hosp} Training set without SMOTE KNN({n_neighbours}) score: {best_knn.score(dict_df_X_train[hosp], dict_df_y_train[hosp])}")
    print(f"{hosp} Test set without SMOTE KNN({n_neighbours}) score: {best_knn.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

    print(f"{hosp} Accuracy without SMOTE KNN({n_neighbours}): {metrics.accuracy_score(dict_df_y_test[hosp], y_pred)}")
    print(hosp, f'KNN({n_neighbours})\n', confusion_matrix(dict_df_y_test[hosp],y_pred))  
    print(hosp, f'KNN({n_neighbours})\n', classification_report(dict_df_y_test[hosp],y_pred))  

### KNN With SMOTE

#### KNN With SMOTE ss=0.1

In [ ]:
n_neighbours = 8
list_smote_sampling = [0.1, 0.5, 1.0]
for hosp in dict_df_X_train:
    for sample in list_smote_sampling:
        sm_ss = SMOTE(random_state=55,sampling_strategy=sample)
        X_train_ss, y_train_ss = sm_ss.fit_sample(dict_df_X_train[hosp], dict_df_y_train[hosp])
        
        # Fit the KNN model with training data and score the test data
        best_knn = KNeighborsClassifier(n_neighbours)    
        best_knn.fit(X_train_ss,y_train_ss)
        #best_knn.score(dict_df_X_test[hosp],dict_df_y_test[hosp])
        
        # save the model to disk
        filename = f'{hosp}_best_knn_{sample}_smote.pkl'
        pickle.dump(best_knn, open(filename, 'wb'))
        
        y_pred = best_knn.predict(dict_df_X_test[hosp])
        # Score for Best KNN, with SMOTE
        print(f"{hosp} Training set {sample} SMOTE KNN({n_neighbours}) score: {best_knn.score(X_train_ss, y_train_ss)}")
        print(f"{hosp} Test set {sample} SMOTE KNN({n_neighbours}) score: {best_knn.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

        print(f"{hosp} Accuracy {sample} SMOTE KNN({n_neighbours}): ", metrics.accuracy_score(dict_df_y_test[hosp], y_pred))
        print(confusion_matrix(dict_df_y_test[hosp],y_pred))  
        print(classification_report(dict_df_y_test[hosp],y_pred)) 